In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random as rand

from sklearn.preprocessing import MinMaxScaler

In [2]:
import os
os.chdir('C:\\code\\activ')

In [3]:
df_name = 'nov_nine_var.xlsx'

In [4]:
df = pd.read_excel('./documents/'+df_name).iloc[:,1::]
X = df.iloc[:,0:21]
y = df.iloc[:,21:22].to_numpy()

In [5]:
y[-1]

array([0.50384917])

In [6]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [7]:
dnn_model = tf.keras.models.load_model('./model/dnn.h5')

In [8]:
def my_loss(real_data, gen_data):
    dist = tf.math.sqrt(tf.reduce_sum(tf.math.square(tf.math.subtract(real_data, gen_data))))
    y_pred = dnn_model(gen_data)

    a = tf.cast(tf.math.subtract(y_pred, 0.50384917), dtype=tf.float32)
    b = tf.cast(dist, dtype=tf.float32)

    loss = tf.math.add(a, b)
    return loss

In [9]:
input_layer = tf.keras.layers.Input(shape=(21,))

encoded = tf.keras.layers.Dense(128, activation='relu')(input_layer)
encoded = tf.keras.layers.Dense(64, activation='relu')(encoded)
encoded = tf.keras.layers.Dense(32, activation='relu')(encoded)
encoded = tf.keras.layers.Dense(16, activation='relu')(encoded)
encoded = tf.keras.layers.Dense(3)(encoded)
 
decoded = tf.keras.layers.Dense(3, activation='relu')(encoded)
decoded = tf.keras.layers.Dense(16, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(32, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(64, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(128, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(21, activation='sigmoid')(decoded)
 
# autoencoder
autoencoder = tf.keras.Model(input_layer, decoded)
 
# encoder
encoder = tf.keras.Model(input_layer, encoded)

# decoder
encoded_input = tf.keras.layers.Input(shape=(3,))

decoded_layer = autoencoder.layers[6](encoded_input)
decoded_layer = autoencoder.layers[7](decoded_layer)
decoded_layer = autoencoder.layers[8](decoded_layer)
decoded_layer = autoencoder.layers[9](decoded_layer)
decoded_layer = autoencoder.layers[10](decoded_layer)
decoded_layer = autoencoder.layers[11](decoded_layer)

decoder = tf.keras.Model(encoded_input, decoded_layer)

In [10]:
autoencoder.compile(optimizer='adam', loss=my_loss, metrics=['mse'])

In [11]:
result = []
pop_result = []
len_data = len(X)
for i in range(1000):
    X_train = X[rand.randint(0, len_data-1)].reshape(-1, 1).T
    X_pred = autoencoder.predict(X_train, verbose=0)

    loss = my_loss(X_train, X_pred)
    autoencoder.train_on_batch(X_pred, X_train)

    pop = dnn_model.predict(X_pred, verbose=0)
    
    result.append(X_pred)
    pop_result.append(pop)

    print(f"epoch: {i}, loss: {loss}, pop: {pop}")

epoch: 0, loss: [[0.93991923]], pop: [[0.47537625]]
epoch: 1, loss: [[0.840945]], pop: [[0.475263]]
epoch: 2, loss: [[1.7428055]], pop: [[0.4752092]]
epoch: 3, loss: [[1.3209826]], pop: [[0.4752423]]
epoch: 4, loss: [[1.4145284]], pop: [[0.47529495]]
epoch: 5, loss: [[1.3420329]], pop: [[0.47538513]]
epoch: 6, loss: [[1.1445074]], pop: [[0.47547638]]
epoch: 7, loss: [[1.0516346]], pop: [[0.47557002]]
epoch: 8, loss: [[1.2713314]], pop: [[0.47579813]]
epoch: 9, loss: [[1.2037107]], pop: [[0.47612837]]
epoch: 10, loss: [[1.6881398]], pop: [[0.47665298]]
epoch: 11, loss: [[1.6959853]], pop: [[0.47625273]]
epoch: 12, loss: [[1.5642595]], pop: [[0.47636592]]
epoch: 13, loss: [[1.5803958]], pop: [[0.47657037]]
epoch: 14, loss: [[1.2174625]], pop: [[0.4769392]]
epoch: 15, loss: [[1.1995945]], pop: [[0.47733477]]
epoch: 16, loss: [[1.0272388]], pop: [[0.47743958]]
epoch: 17, loss: [[0.912765]], pop: [[0.4779832]]
epoch: 18, loss: [[1.3774161]], pop: [[0.47845823]]
epoch: 19, loss: [[0.6535958]

In [12]:
gen = autoencoder.predict(X)

14/14 [==============================] - 0s 1ms/step


In [13]:
print(dnn_model.predict(gen, verbose=0).max())
print(dnn_model.predict(gen, verbose=0).min())
print(dnn_model.predict(gen, verbose=0).mean())

0.49221352
0.44438833
0.45962927


In [14]:
latest = np.array([X[-1]])
latest_gen = autoencoder.predict(latest)
dist = np.sqrt(np.sum(np.square(latest-latest_gen)))

print(dist)

1/1 [==============================] - 0s 24ms/step
2.633794827101483


In [15]:
result = np.array(result).reshape(1000, 21)
X_result = scaler.inverse_transform(result)
pop_result = np.array(pop_result).reshape(1000, 1)

In [16]:
df = np.append(X_result, pop_result, axis=1)

In [17]:
pd_df = pd.DataFrame(df)
pd_df.to_excel('./documents/result/ae_formula.xlsx')